# EDAN20 - LAB 0

Language Technology EDAN20 @ LTH - http://cs.lth.se/edan20/coursework/laboratory-0/

Author Jonatan Kronander

In [39]:
import regex as re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [40]:
sum(WORDS.values())

1115585

In [41]:
correction('speling')

'spelling'

## 1. Selection Mechanism: argmax 
We choose the candidate with the highest combined probability.


In Python, max with a key argument does 'argmax'.

## 2.Candidate Model: c ∈ candidates
This tells us which candidate corrections, c, to consider.


In [25]:
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)] #Split to two words
    deletes    = [L + R[1:]               for L, R in splits if R] #Remove one letter
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1] #Swap two adjacent letters
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters] #Change one letter to another
    inserts    = [L + c + R               for L, R in splits for c in letters] #Add a letter
    return set(deletes + transposes + replaces + inserts)

This can be a big set. For a word of length n, there will be n deletions, n-1 transpositions, 26n alterations, and 26(n+1) insertions, for a total of 54n+25 (of which a few are typically duplicates). For example,

In [26]:
len(edits1('somthing'))

442

## 3.Language Model: P(c)  
The probability that c appears as a word of English text. 
For example, occurrences of "the" make up about 7% of English text, so we should have P(the) = 0.07.



## 4. Error Model: P(w|c) 
The probability that w would be typed in a text when the author meant c. 
For example, P(teh|the) is relatively high, but P(theeexyz|the) would be very low.